In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

In [11]:
# read the merged_data.csv file
df = pd.read_csv('merged_data.csv')

In [12]:
df

,type,geometry,density
0,Noise - Park - Loud Music/Party,"(40.74217591632833, -73.8041140918047)",0.000000
1,Noise - Residential - Loud Music/Party,"(40.66677169641655, -73.89130158867503)",0.000000
2,Noise - Street/Sidewalk - Loud Talking,"(40.709122186956755, -73.94854691873594)",0.000000
3,Noise - Residential - Loud Music/Party,"(40.842728168449554, -73.84687754834488)",7548.748498
4,Noise - Residential - Loud Music/Party,"(40.70651829396309, -73.90983347440181)",0.000000
...,...,...,...
6057173,building,"POLYGON ((-74.1566219 40.8335072, -74.1574011 ...",NaN
6057174,building,"POLYGON ((-74.1448735 40.8355725, -74.1452398 ...",NaN
6057175,school,"POLYGON ((-74.0667017 40.791092, -74.0667022 4...",NaN
6057176,apartments,"POLYGON ((-74.1516162 40.7368786, -74.1516162 ...",NaN


In [13]:
# convert the 'geometry' column to a geopandas dataframe
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt
import ast


# 定义一个函数来处理不同类型的 geometry 数据
def convert_geometry(geom):
    if pd.isna(geom):
        return None  # 忽略空值
    if isinstance(geom, str):
        try:
            if geom.startswith('POLYGON') or geom.startswith('POINT'):
                # 处理 WKT 格式
                return wkt.loads(geom)
            elif geom.startswith('(') and geom.endswith(')'):
                # 处理 (lat, lon) 格式
                coords = ast.literal_eval(geom)
                if isinstance(coords, tuple) and len(coords) == 2:
                    # 确保是有效的坐标对（浮点或整数类型）
                    if isinstance(coords[0], (int, float)) and isinstance(coords[1], (int, float)):
                        return Point(coords)
        except (ValueError, SyntaxError):
            return None
    return None

# 应用转换函数
df['geometry'] = df['geometry'].apply(convert_geometry)

# 过滤掉转换失败的条目
df = df[df['geometry'].notna()]

# 创建 GeoDataFrame，设置 crs 为 EPSG:4326（通用地理坐标系）
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')




In [17]:
gdf

,type,geometry,density,hex_id
0,Noise - Park - Loud Music/Party,POINT (40.74218 -73.80411),0.000000,8af05aa41747fff
1,Noise - Residential - Loud Music/Party,POINT (40.66677 -73.89130),0.000000,8af05aa65a07fff
2,Noise - Street/Sidewalk - Loud Talking,POINT (40.70912 -73.94855),0.000000,8af05ab4c75ffff
3,Noise - Residential - Loud Music/Party,POINT (40.84273 -73.84688),7548.748498,8af05aa44cd7fff
4,Noise - Residential - Loud Music/Party,POINT (40.70652 -73.90983),0.000000,8af05ab4b08ffff
...,...,...,...,...
6057173,building,"POLYGON ((-74.15662 40.83351, -74.15740 40.833...",NaN,None
6057174,building,"POLYGON ((-74.14487 40.83557, -74.14524 40.835...",NaN,None
6057175,school,"POLYGON ((-74.06670 40.79109, -74.06670 40.791...",NaN,None
6057176,apartments,"POLYGON ((-74.15162 40.73688, -74.15162 40.736...",NaN,None


In [18]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely import wkt
import ast
import h3

# 假设你已经有一个 GeoDataFrame，名为 gdf
# 创建 gdf（前面步骤已经完成）

# 定义一个函数，将 geometry 转换为 H3 索引，分辨率为 10
def geometry_to_h3(geometry, resolution=10):
    if isinstance(geometry, Point):
        # 如果是 Point 对象，直接使用其坐标
        lat, lon = geometry.y, geometry.x
        return h3.geo_to_h3(lat, lon, resolution)
    elif isinstance(geometry, Polygon):
        # 如果是 Polygon 对象，使用其质心来计算 H3 索引
        centroid = geometry.centroid
        lat, lon = centroid.y, centroid.x
        return h3.geo_to_h3(lat, lon, resolution)
    else:
        return None

# 应用函数并创建一个新的 H3 列
gdf['h3_index'] = gdf['geometry'].apply(geometry_to_h3)





AttributeError: module 'h3' has no attribute 'geo_to_h3'